In [1]:
Group Members 11: Jerry Yan, Annie Tsoromocos, Siddh Patel, Miles Da Silva
Github links:
https://github.com/owid/covid-19-data/tree/master/public/data/ , 

https://github.com/CSSEGISandData/COVID-19 , 

https://www150.statcan.gc.ca/n1/pub/13-26-0003/132600032020001-eng.htm 


Title: 
What correlations 
How strong the relationship is between vaccinations and hospitalization rates

Introduction:

Provide some relevant background information on the topic so that someone unfamiliar with it will be prepared to understand the rest of your proposal
Clearly state the question you will try to answer with your project
Identify and describe the dataset that will be used to answer the question
Preliminary exploratory data analysis:
Demonstrate that the dataset can be read from the web into R 
Clean and wrangle your data into a tidy format
Using only training data, summarize the data in at least one table (this is exploratory data analysis). An example of a useful table could be one that reports the number of observations in each class, the means of the predictor variables you plan to use in your analysis and how many rows have missing data. 
Using only training data, visualize the data with at least one plot relevant to the analysis you plan to do (this is exploratory data analysis). An example of a useful visualization could be one that compares the distributions of each of the predictor variables you plan to use in your analysis.
Methods:


Explain how you will conduct either your data analysis and which variables/columns you will use. Note - you do not need to use all variables/columns that exist in the raw data set. In fact, that's often not a good idea. For each variable think: is this a useful variable for prediction?
Describe at least one way that you will visualize the results
Expected outcomes and significance:


What do you expect to find?
What impact could such findings have?
What future questions could this lead to?



ERROR: Error in parse(text = x, srcfile = src): <text>:1:7: unexpected symbol
1: Group Members
          ^


In [2]:
library(tidyverse)
covid_data <- read_csv("https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv")
covid_data

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

Warning message:
“package ‘ggplot2’ was built under R version 4.0.1”
Warning message:
“package ‘tibble’ was built under R version 4.0.2”
Warning message:
“package ‘tidyr’ was built under R version 4.0.2”
Warning message:
“package ‘dplyr’ was built under R version 4.0.2”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Parsed with column specification:
cols(
  .default = col_double(),
  iso_code = col_character(),
  continent = col_character(),
  location = col_character(),
  date = col_date(format = ""),
  icu_patients = col_logical(),
  icu_patients_per_million = col_logical(),
  hosp_patients = col_logical(),
  hosp_patients_per_million = col_logical(),

iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,⋯,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
<chr>,<chr>,<chr>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<lgl>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<lgl>,<lgl>,<lgl>
AFG,Asia,Afghanistan,2020-02-24,5,5,NA,NA,NA,NA,⋯,NA,NA,37.746,0.5,64.83,0.511,NA,NA,NA,NA
AFG,Asia,Afghanistan,2020-02-25,5,0,NA,NA,NA,NA,⋯,NA,NA,37.746,0.5,64.83,0.511,NA,NA,NA,NA
AFG,Asia,Afghanistan,2020-02-26,5,0,NA,NA,NA,NA,⋯,NA,NA,37.746,0.5,64.83,0.511,NA,NA,NA,NA
AFG,Asia,Afghanistan,2020-02-27,5,0,NA,NA,NA,NA,⋯,NA,NA,37.746,0.5,64.83,0.511,NA,NA,NA,NA
AFG,Asia,Afghanistan,2020-02-28,5,0,NA,NA,NA,NA,⋯,NA,NA,37.746,0.5,64.83,0.511,NA,NA,NA,NA
AFG,Asia,Afghanistan,2020-02-29,5,0,0.714,NA,NA,NA,⋯,NA,NA,37.746,0.5,64.83,0.511,NA,NA,NA,NA
AFG,Asia,Afghanistan,2020-03-01,5,0,0.714,NA,NA,NA,⋯,NA,NA,37.746,0.5,64.83,0.511,NA,NA,NA,NA
AFG,Asia,Afghanistan,2020-03-02,5,0,0.000,NA,NA,NA,⋯,NA,NA,37.746,0.5,64.83,0.511,NA,NA,NA,NA
AFG,Asia,Afghanistan,2020-03-03,5,0,0.000,NA,NA,NA,⋯,NA,NA,37.746,0.5,64.83,0.511,NA,NA,NA,NA
